In [5]:
from aocd.models import Puzzle
import itertools
from functools import cache, partial
from multiprocessing import Pool
import re
puzzle = Puzzle(year=2023, day=12)
data = puzzle.input_data
%run helper.ipynb
# data = """???.### 1,1,3
# .??..??...?##. 1,1,3
# ?#?#?#?#?#?#?#? 1,3,1,6
# ????.#...#... 4,1,1
# ????.######..#####. 1,6,5
# ?###???????? 3,2,1"""

In [10]:
lookup = [".","#"]

PRINT = False

def cprint(*args, **kwargs):
    if PRINT:
        print(*args, **kwargs)

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def min_len(groups):
    return sum(groups) + len(groups) - 1

@cache
def solve_single(spring_line, groups):
    if len(groups) == 0:
        if "#" not in spring_line:
            return 1
        else:
            return 0
    
    diff = len(spring_line) - min_len(groups)
    return solve_single_helper(spring_line, groups, diff, "")

def solve_simple()

@cache
def solve_single_helper(spring_line, groups, diff, current):
    if len(groups) == 0:
        if len(spring_line) == len(current):
            cprint(current)
            return 1
        return 0
    total = 0
    for i in range(diff+1):
        s = current + "."*i + "#"*groups[0]
        if len(groups) == 1:
            s = s + "." * (diff - i)
        else:
            s = s + "."
        indexes = find(spring_line[:len(s)], "#")
        if all([s[i] == "#" for i in indexes]):
            c = solve_single_helper(spring_line, groups[1:], diff-i, s)
            total += c
    return total

def count_poss(bad_springs, groups, depth):
    spacing = "    " * depth
    cprint(spacing, "Args: ", bad_springs, groups)
    if len(groups) == 0:
        if len(bad_springs) != 0:
            if "#" in "".join(bad_springs):
                cprint(spacing, "Zero failure")
                return 0
            else:
                return 1
        else:
            return 1
    else:
        if len(bad_springs) == 0:
            return 0
    if min_len(groups) > sum([len(b)+1 for b in bad_springs]):
        cprint(spacing, "Not enough")
        return 0
    total = 0
    for i in range(len(groups)+1):
        if min_len(groups[:i]) > len(bad_springs[0]):
            cprint(spacing, f"Stopping on {i} due to length")
            break
        cprint(spacing, "  Trying" + str((bad_springs[0], groups[:i])))
        remaining_count = count_poss(bad_springs[1:], groups[i:], depth+1)
        if remaining_count != 0:
            cprint(spacing, "solving", bad_springs[0], groups[:i])
            single = solve_single(bad_springs[0], groups[:i])
            total += (single * remaining_count)
    return total
    
class Row(object):
    def __init__(self, s, expand=False):
        self.original = s
        springs, groups = s.split(" ")
        self.springs = springs
        self.groups = list(map(int, groups.split(",")))
        if expand:
            self.springs = "?".join([self.springs] * 5)
            self.groups = self.groups * 5
        self.bad_springs = re.findall(r"[#?]+", self.springs)
    def count(self):
        return count_poss(tuple(self.bad_springs), tuple(self.groups), 0)

In [11]:
rs = [Row(s).count() for s in data.split("\n")]

In [12]:
puzzle.answer_a = sum(rs)

In [13]:
rs2 = [Row(s, True) for s in data.split("\n")]

In [ ]:
for i in range(len(rs2)):
    c = count_poss(tuple(rs2[i].bad_springs), tuple(rs2[i].groups), 0)
    print(rs2[i].original, "=====", c)

In [150]:
count_poss(("??", "???#?", "?"), (1,4), 0)

4